## Load the SentenceTransformer and test the embedding is working


In [5]:
from sentence_transformers import SentenceTransformer

embeddingMModel = SentenceTransformer('all-MiniLM-L6-v2')

# def embed_text(text):
#     return embeddingModel.encode(text)
# # Example usage
# if __name__ == "__main__":
#     sample_text = "This is a sample text for embedding."
#     embedding = embed_text(sample_text)
#     print(f"Embedding for the sample text: {embedding[:5]}")

### Load Document and spilt into chunks and then embedding the chunks

In [6]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# load the pdf document
loader = PyPDFLoader("dataset/AI Agents guidebook.pdf")
documents = loader.load()

# split the document into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=300)
chunks = text_splitter.split_documents(documents)
print(f"Number of chunks created: {len(chunks)}")

Ignoring wrong pointing object 899 0 (offset 0)


Number of chunks created: 122


### Embed the chunks 

In [7]:
encoded_chunks = [embeddingMModel.encode(chunk.page_content) for chunk in chunks]
print(f"Embedding for first chunk: {encoded_chunks[0][:5]}")
print(f'Total number of encoded chunks: {len(encoded_chunks)}')
print(f'Length of each encoded chunk: {len(encoded_chunks[1])}')
print(f'Embedding Model Shape: {encoded_chunks[0].shape}')
# Example usage
# if __name__ == "__main__":
#     print(f"Number of chunks created: {len(chunks)}")
#     print(f"Embedding for first chunk: {encoded_chunks[0][:5]}")    

Embedding for first chunk: [-0.05577217 -0.04005972  0.02752373  0.03279826  0.05366068]
Total number of encoded chunks: 122
Length of each encoded chunk: 384
Embedding Model Shape: (384,)


### Save the embedding in Chromdb

In [8]:
import chromadb

# Create a ChromaDB client
client = chromadb.Client()
# Create a collection to store the embeddings
collection = client.get_or_create_collection(name="ai_agents_guidebook")
# Add the chunks , their embeddings and metadata to the collection
for i, chunk in enumerate(chunks):
    collection.add(
        documents=[chunk.page_content],
        embeddings=[encoded_chunks[i]],
        ids=[str(i)],
        metadatas=[{"source": "AI Agents guidebook", "chunk_index": i}]
    )
print(f"Embeddings saved to ChromaDB collection 'ai_agents_guidebook'. {collection.count()} items added.")
print(f"Number of chunks created: {collection}")

Embeddings saved to ChromaDB collection 'ai_agents_guidebook'. 122 items added.
Number of chunks created: Collection(name=ai_agents_guidebook)


### Query and retrieve the vector db here. it is chromadb... Testing done before using LLM


In [9]:
# create a method which will take query and return the relevant chunks
def query_chromadb(query, top_k=3):
    # Embed the query using the same embedding model
    query_embedding = embeddingMModel.encode(query)
    # Query the collection for similar embeddings
    results = collection.query(
        query_embeddings=[query_embedding],
        n_results=top_k
    )
    return results

       
# Test the method using Query list before using LLM
query_list = [
    "What is MCP Server",
    "What is Kayak Tool?",
    "Can i work from home today?",
    "How to use AutoGPT?",
]



# Example usage with the query list
if __name__ == "__main__":
    print("=" * 60)
    print("🔍 TESTING SEMANTIC SEARCH")
    print("=" * 60)
    for query in query_list:
        print(f"Results for query: {query}")
        results = query_chromadb(query, top_k=3)
        for i, (doc, score, metadata) in enumerate(zip(results['documents'][0], results['distances'][0], results['metadatas'][0])):
            print(f"\n  Result {i+1}:")
            print(f"  📄 Topic: {metadata['source']}")
            print(f"  📏 Distance: {score:.4f} (lower = more similar)")
            print(f"  📖 Text: {doc[:50]}...")

🔍 TESTING SEMANTIC SEARCH
Results for query: What is MCP Server

  Result 1:
  📄 Topic: AI Agents guidebook
  📏 Distance: 0.8619 (lower = more similar)
  📖 Text: DailyDoseofDS.com 
Finally, once we have all the a...

  Result 2:
  📄 Topic: AI Agents guidebook
  📏 Distance: 0.9035 (lower = more similar)
  📖 Text: DailyDoseofDS.com 
 
#8) Integrate MCP server with...

  Result 3:
  📄 Topic: AI Agents guidebook
  📏 Distance: 0.9076 (lower = more similar)
  📖 Text: DailyDoseofDS.com 
#7) Integrate MCP server with C...
Results for query: What is Kayak Tool?

  Result 1:
  📄 Topic: AI Agents guidebook
  📏 Distance: 0.5971 (lower = more similar)
  📖 Text: DailyDoseofDS.com 
 
#4) Kayak tool 
A custom Kaya...

  Result 2:
  📄 Topic: AI Agents guidebook
  📏 Distance: 0.6021 (lower = more similar)
  📖 Text: DailyDoseofDS.com 
 
#4) Kayak tool 
A custom Kaya...

  Result 3:
  📄 Topic: AI Agents guidebook
  📏 Distance: 1.0708 (lower = more similar)
  📖 Text: DailyDoseofDS.com 
#5) Browserbase Tool

## Initlaise the SQLite for storing the response generated by LLM

In [10]:
import sqlite3



### Create the Prompt with User Query 

In [11]:
system_prompt_context = """
You are an AI assistant helping users find information in a document. Use the following pieces of context to answer the question at the end. 
If you don't know the answer, just say that you don't know.
"""
def create_prompt(user_query, context_chunks):
    context_text = "\n\n".join([f"Context {i+1}:\n{chunk}" for i, chunk in enumerate(context_chunks)])
    prompt = f"{system_prompt_context}\n\n{context_text}\n\nQuestion: {user_query}\nAnswer:"
    return prompt

if __name__ == "__main__":
    user_query = "What is MCP Server?"
    results = query_chromadb(user_query, top_k=3)
    # print(f'Results Retrieved: {results}')
    context_chunks = results['documents'][0]
    # print(f'Context Chunks Retrieved: {len(context_chunks)}')
    # print(f'First Context Chunk: {context_chunks[2]}...\n\n')
    prompt = create_prompt(user_query, context_chunks)
    print("Generated Prompt:")
    print(prompt)   


Generated Prompt:

You are an AI assistant helping users find information in a document. Use the following pieces of context to answer the question at the end. 
If you don't know the answer, just say that you don't know.


Context 1:
DailyDoseofDS.com 
Finally, once we have all the agents and tools deﬁned we set up and kickoﬀ our 
deep researcher crew. 
 
#7) Create MCP Server 
Now, we'll encapsulate our deep research team within an MCP tool. With just a 
few lines of code, our MCP server will be ready. 
Let's see how to connect it with Cursor. 
79

Context 2:
DailyDoseofDS.com 
 
#8) Integrate MCP server with Cursor 
Go to: File → Preferences → Cursor Settings → MCP → Add new global MCP 
server 
In the JSON ﬁle, add what's shown below  
80

Context 3:
DailyDoseofDS.com 
#5) Setup Crew and Kickoﬀ 
We set up and kick oﬀ our ﬁnancial analysis crew to get the result shown below! 
 
#6) Create MCP Server 
Now, we encapsulate our ﬁnancial analyst within an MCP tool and add two more 
tools t

### Call the Groq and OpenAI  LLM Models

In [12]:
from langchain_openai import ChatOpenAI
from langchain_google_genai import ChatGoogleGenerativeAI

from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
import os
from dotenv import load_dotenv

load_dotenv()
# Initialize LLMs

google_api_key = os.getenv("GOOGLE_API_KEY")
google_llm = None
google_llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash-lite", temperature=0, google_api_key=google_api_key)

openai_api_key = os.getenv("OPENAI_API_KEY")
openai_llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0, openai_api_key=openai_api_key)

groq_api_key = os.getenv("GROQ_API_KEY")
groq_llm = ChatGroq(model="llama-3.3-70b-versatile", temperature=0, groq_api_key=groq_api_key)

## Pass the query and context to create a response from LLM

In [13]:
def create_prompt(user_query, context_chunks):
    context_text = "\n\n".join([f"Context {i+1}:\n{chunk}" for i, chunk in enumerate(context_chunks)])
    prompt = f"{system_prompt_context}\n\n{context_text}\n\nQuestion: {user_query}\nAnswer:"
    return prompt

def generate_openai_response(prompt):
    response = openai_llm.invoke(prompt)
    print(f'\n\n Open AI Response: {response.content} *80')
    return response.content

def generate_google_response(prompt):
    response = google_llm.invoke(prompt)
    print(f'\n\n Google LLM Response: {response.content}')
    return response.content

def generate_groq_response(prompt):
    response = groq_llm.invoke(prompt)
    print(f'\n\n Groq LLM Response: {response.content}')
    return response.content



user_query = "Tell me more about multi agent team researcher?"
results = query_chromadb(user_query, top_k=3)

# print(f'Results Retrieved: {results}')
context_chunks = results['documents'][0]
# print(f'Context Chunks Retrieved: {len(context_chunks)}')
# print(f'First Context Chunk: {context_chunks[2]}...\n\n')
prompt = create_prompt(user_query, context_chunks)
# print("Generated Prompt:")
# print(prompt)  

# Generate responses from both LLMs
gemini_llmresponse = generate_google_response(prompt)
groq_response = generate_groq_response(prompt)
               




 Google LLM Response: The context describes a "Multi-agent Deep Researcher" that functions similarly to ChatGPT's deep research feature, providing detailed insights on any topic. It can be built as a local alternative using a specific tech stack. The workflow involves a user submitting a query, a web search agent performing a deep web search, a research analyst verifying and deduplicating the results, and a technical writer crafting a coherent response with citations. This system utilizes multiple agents working together, with one agent for research and another for analysis and writing.


 Groq LLM Response: Based on the provided context, a multi-agent team researcher appears to be a system where multiple agents work together to provide detailed insights on a topic. 

From Context 2, it seems that such a system can be built using a tech stack that includes a platform for deep web research, multi-agent orchestration, and a local server for deep research. The workflow involves:

1. A u

## Save the final Syntentized Response in sqlite

In [14]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
import numpy as np

synthesis_prompt = ChatPromptTemplate.from_template("""
Synthesize these two answers into one comprehensive response:

Answer from Gemini LLM:
{response1}

Answer from Groq Llama:
{response2}

Original Question: {question}

Instructions:
1. Combine the best insights from both answers
2. Avoid redundancy
3. Provide a complete, well-organized answer
4. Maintain accuracy and clarity

Provide a single synthesized answer that is better and more complete than both.
""")

chain_synthesis = synthesis_prompt | google_llm | StrOutputParser()
final_response = chain_synthesis.invoke({
    "response1": gemini_llmresponse,
    "response2": groq_response,
    "question": user_query,

})

print(f"✓ Synthesized Final Response:\n{final_response}")

# create a database connection
sqlite_db_conn = sqlite3.connect("documents.db")

# Save it to SQLite
cursor = sqlite_db_conn.cursor()

# 2. Create a table
cursor.execute("""
CREATE TABLE IF NOT EXISTS llm_response (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    user_query TEXT,
    vector_embedding BLOB,
    llmresponse TEXT
)
""")
# convert the query_vectorbinding into a BLOB using numpy array
queryEmbedding = embeddingMModel.encode(user_query)
print(f'\n\n embedd query is {queryEmbedding[0]}')

embeddingBlob = np.array(queryEmbedding,dtype=np.float32).tobytes()


# 3. Insert a record
cursor.execute("""
INSERT INTO llm_response (user_query, vector_embedding, llmresponse)
VALUES (?, ?, ?)
""", (user_query, embeddingBlob, final_response))

# 4. Commit changes
sqlite_db_conn.commit()

# 5. Verify vector embedding by decode
cursor.execute(
    "SELECT vector_embedding FROM llm_response WHERE user_query = ?",
    (user_query,)
)
blob = cursor.fetchone()[0]

vectorFetchedVal = np.frombuffer(blob, dtype=np.float32)
print(f'Fetched vector binding {vectorFetchedVal[0]}')


print("\n\n Closing Database connection")
sqlite_db_conn.close()



✓ Synthesized Final Response:
A "Multi-agent Deep Researcher" or "multi-agent team researcher" is a system designed to provide detailed insights on any given topic, functioning analogously to advanced deep research features found in platforms like ChatGPT. This system is built upon the principle of **collaboration between multiple specialized agents**, each performing a distinct role to achieve a comprehensive and well-structured output.

The core workflow of such a system typically involves the following steps:

1.  **User Query Submission:** The process begins with a user submitting a specific query or topic of interest.
2.  **Deep Web Search:** A dedicated **web search agent** is deployed to conduct an extensive search across the deep web, gathering a broad range of relevant information.
3.  **Research Analysis and Verification:** A **research analyst agent** then takes over to meticulously verify the gathered information, deduplicate redundant findings, and ensure the accuracy and 

## Shorten the summary to 280 characters

In [15]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# Create a summarization prompt to condense the response to 280 characters
summarization_prompt = ChatPromptTemplate.from_template("""
Summarize the following text in exactly 280 characters or less. 
Make it concise, clear, and informative. Preserve the key message.

Text to summarize:
{text}

Important: Your summary must be exactly 280 characters or less (including spaces and punctuation).
Respond with ONLY the summarized text, no additional explanation.
""")

# Use Google LLM to create the 280-character summary
chain_summarize = summarization_prompt | google_llm | StrOutputParser()

shortened_response = chain_summarize.invoke({
    "text": final_response
})

# Ensure the response doesn't exceed 280 characters (fallback truncation)
if len(shortened_response) > 280:
    shortened_response = shortened_response[:277] + "..."

print(f"\n{'='*80}")
print(f"✓ AI-Generated Summary (280 characters max):")
print(f"{'='*80}")
print(f"{shortened_response}")
print(f"\nOriginal length: {len(final_response)} characters")
print(f"Summary length: {len(shortened_response)} characters")


✓ AI-Generated Summary (280 characters max):
A Multi-agent Deep Researcher uses specialized agents to collaborate on user queries. A search agent gathers info, an analyst verifies it, and a writer generates a coherent, cited response. This division of labor ensures efficient, thorough, and high-quality research insights.

Original length: 2142 characters
Summary length: 277 characters


### Send the summarized summary as a email 

In [38]:
from sendgrid import SendGridAPIClient
from sendgrid.helpers.mail import Mail
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.application import MIMEApplication
import os
import pandas as pd
load_dotenv()
load_dotenv()

def send_email_with_sendgrid(summarized_text, recipient_email="cloudwizkid1@gmail.com"):
    """
    Send the summarized response via email using SendGrid API
    """
    # Get SendGrid API key from environment
    sendgrid_api_key = os.getenv("SENDGRID_API_KEY")
    sender_email = "cloudwizkid1@gmail.com"
    
    if not sendgrid_api_key:
        print("❌ Error: SENDGRID_API_KEY not found in .env file")
        return False
    
    try:
        # Create HTML email body
        html_body = f"""
        <html>
          <body style="font-family: Arial, sans-serif; line-height: 1.6; color: #333;">
            <div style="max-width: 600px; margin: 0 auto; padding: 20px; border: 1px solid #ddd; border-radius: 5px;">
              <h2 style="color: #4CAF50;">✓ AI-Generated Summary</h2>
              
              <p><strong>Topic:</strong> Multi-Agent Team Researcher</p>
              
              <div style="background-color: #f5f5f5; padding: 15px; border-left: 4px solid #4CAF50; margin: 20px 0;">
                <p>{summarized_text}</p>
              </div>
              
              <p><strong>Summary Statistics:</strong></p>
              <ul>
                <li>Character Length: {len(summarized_text)}/280</li>
                <li>Generated using: Google Generative AI (Gemini)</li>
                <li>Timestamp: {pd.Timestamp.now()}</li>
              </ul>
              
              <hr>
              <p style="font-size: 12px; color: #666;">This is an automated email from AI Capstone Assignment. Please do not reply to this email.</p>
            </div>
          </body>
        </html>
        """
        
        # Create plain text version
        text_body = f"""
        AI-Generated Summary
        ====================
        
        Topic: Multi-Agent Team Researcher
        
        {summarized_text}
        
        Character Length: {len(summarized_text)}/280
        Generated using: Google Generative AI (Gemini)
        """
        
        # Send email via SendGrid API
        print("\n📧 Sending email via SendGrid...")
        message = Mail(
            from_email=sender_email,
            to_emails=recipient_email,
            subject="AI-Generated Summary: Multi-Agent Team Researcher",
            plain_text_content=text_body,
            html_content=html_body
        )
        
        sg = SendGridAPIClient(sendgrid_api_key)
        response = sg.send(message)
        
        print(f"✓ Email sent successfully to {recipient_email}")
        return True
        
    except Exception as e:
        print(f"❌ Error sending email: {e}")
        return False

# Send the summarized response
send_email_with_sendgrid(shortened_response,"cloudwizkid1@gmail.com")


📧 Sending email via SendGrid...
❌ Error sending email: HTTP Error 401: Unauthorized


False

### Load and authenticate twilio configurations to send whatsapp and sms

In [18]:
import os
from dotenv import load_dotenv
load_dotenv()
twilio_account_sid = os.getenv("TWILIO_ACCOUNT_SID")
twilio_auth_token = os.getenv("TWILIO_AUTH_TOKEN")
twilio_phone_number = os.getenv("TWILIO_PHONE_NUMBER")
whatsapp_from = os.getenv("TWILIO_WHATSAPP_NUMBER", "whatsapp:+14155238886")
#print(f'Twilio SID: {twilio_account_sid}, Token: {twilio_auth_token}, Phone: {twilio_phone_number}')
from twilio.rest import Client

client = Client(twilio_account_sid, twilio_auth_token)

def send_sms_via_twilio(to_phone_number, message_body):
    try:
        message = client.messages.create(
            body=message_body,
            from_=twilio_phone_number,
            to=to_phone_number
        )
        print(f"✓ SMS sent successfully to {to_phone_number}. Message SID: {message.sid}")
        return True
    except Exception as e:
        print(f"❌ Error sending SMS: {e}")
        return False
# Send the summarized response via SMS
send_sms_via_twilio("+918860811855", shortened_response)

# send whatsapp message via twilio

def send_whatsapp_via_twilio(to_phone_number, message_body):
    try:
        message = client.messages.create(
            body=message_body,
            from_=whatsapp_from,
            to='whatsapp:' + to_phone_number
        )
        print(f"✓ WhatsApp message sent successfully to {to_phone_number}. Message SID: {message.sid}")
        return True
    except Exception as e:
        print(f"❌ Error sending WhatsApp message: {e}")
        return False
# Send the summarized response via WhatsApp
send_whatsapp_via_twilio("+918860811855", shortened_response)

✓ SMS sent successfully to +918860811855. Message SID: SMa7118e0234c1614013d8f0050dacb340
✓ WhatsApp message sent successfully to +918860811855. Message SID: SM6888e817485bf8aad9e1f039d5b3507a


True

### Send the summarized response in a Tweet

In [31]:
load_dotenv()
import requests
import json
from urllib.parse import unquote

# Get Bearer Token for Twitter API v2 and decode if URL-encoded
twitter_bearer_token = os.getenv("TWITTER_BEARER_TOKEN")
if twitter_bearer_token:
    twitter_bearer_token = unquote(twitter_bearer_token)

def send_tweet(tweet_text):
    """
    Send tweet using Twitter API v2 with Bearer Token (HTTP request)
    """
    if not twitter_bearer_token:
        print("❌ Error: TWITTER_BEARER_TOKEN not found in .env file")
        return False
    
    try:
        # Twitter API v2 endpoint
        url = "https://api.twitter.com/2/tweets"
        
        # Prepare headers with Bearer Token
        headers = {
            "Authorization": f"Bearer {twitter_bearer_token}",
            "Content-Type": "application/json"
        }
        
        # Prepare payload
        payload = {"text": tweet_text}
        
        # Make POST request
        response = requests.post(url, json=payload, headers=headers)
        
        # Check if tweet was created successfully (201 = Created)
        if response.status_code == 201:
            data = response.json()
            print(f"✓ Tweet posted successfully!")
            print(f"Tweet ID: {data['data']['id']}")
            print(f"Tweet text: {tweet_text[:100]}...")
            return True
        elif response.status_code == 401:
            print("❌ Error: Bearer Token is invalid or expired")
            print("Full Response:", response.text)
            return False
        elif response.status_code == 403:
            print("❌ Error: 403 Forbidden - your app may not have Write permissions")
            print("Full Response:", response.text)
            print("\nTo fix this:")
            print("1. Go to https://developer.twitter.com/en/portal/dashboard")
            print("2. Select your app and go to App Settings")
            print("3. Under 'User authentication settings', ensure Permissions are set to 'Read and Write'")
            print("4. Regenerate your tokens after changing permissions")
            return False
        else:
            print(f"❌ Error: HTTP {response.status_code}")
            print("Full Response:", response.text)
            return False
            
    except requests.exceptions.RequestException as e:
        print(f"❌ Request error: {e}")
        return False
    except Exception as e:
        print(f"❌ Unexpected error: {e}")
        return False

# Send the summarized response via Tweet
send_tweet(shortened_response)

❌ Error: 403 Forbidden - your app may not have Write permissions
Full Response: {
  "title": "Unsupported Authentication",
  "detail": "Authenticating with OAuth 2.0 Application-Only is forbidden for this endpoint.  Supported authentication types are [OAuth 1.0a User Context, OAuth 2.0 User Context].",
  "type": "https://api.twitter.com/2/problems/unsupported-authentication",
  "status": 403
}

To fix this:
1. Go to https://developer.twitter.com/en/portal/dashboard
2. Select your app and go to App Settings
3. Under 'User authentication settings', ensure Permissions are set to 'Read and Write'
4. Regenerate your tokens after changing permissions


False